In [ ]:
#Compared to v1:

# Added demo comparisons of the share of consumers living paycheck to paycheck

## Next up, share struggling to pay bills, average savings, average credit score, share making payments related to credit products

# Q&A calculator


## Setting up the Lending Club time comparisons

First: share living paycheck to paycheck by time

**Importing libraries, loading csv**

In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
dir_path = 'C:/Users/marco/Dropbox (MPD)/Analytics Argentina/Billable products/Surveys/Lending Club/All RDs'

# Change the working directory to the specified path
os.chdir(dir_path)

In [3]:
# Needs to be updated periodically

current_month = "April, 2023"
current_order = ['November 2021','December 2021','January 2022','February 2022', 'March 2022','April 2022',
                 'May 2022', "June 2022", "July 2022", "August 2022", "September 2022", "October 2022",
                 "November 2022", "December 2022", "January 2023",'February 2023',"March, 2023","April, 2023"]

In [4]:
#Setting up dataframe + codebook

df=pd.read_csv('LC18.csv', low_memory=False) #harcoded reference

#double header, will create a codebook but for now im just keeping the code and not the description
df=df.drop(0)
# it imported some whitenoise, as all responses must have a startdate
df=df[df["StartDate"].notnull()]
 # No test responses considered
df=df[df["Status"]=="IP Address"]

print(f"Load successful. {df.shape[0]} rows and {df.shape[1]} columns loaded into dataframe")

codebook=pd.read_csv('LC18.csv', low_memory=False)
codebook=codebook.loc[0].to_dict()

Load successful. 3652 rows and 277 columns loaded into dataframe


### Pre processing

In [5]:
# Need to format dates because they got imported as numbers

date_columns=['StartDate','EndDate']
for col in date_columns:
    df[col] = df[col].astype(float)
    df[col] = df[col].astype(int)
    df[col]= df[col]+ 693594 # Offset for converting to a valid date in the Gregorian calendar
    df[col] = df[col].apply(lambda x: datetime.fromordinal(x))
    df[col] = df[col].apply(lambda x: x.strftime('%Y-%m-%d'))
    
print('Check dates: look ok?')
print(np.sort(df.EndDate.unique()))

Check dates: look ok?
['2023-04-03' '2023-04-04' '2023-04-05' '2023-04-06' '2023-04-07'
 '2023-04-08' '2023-04-09' '2023-04-10' '2023-04-11' '2023-04-12'
 '2023-04-13' '2023-04-14' '2023-04-15' '2023-04-16' '2023-04-17']


In [6]:
# Need to format weight as a float

df.weight=df.weight.astype(float)
print("Weight sum should be 260M or similar")
print(df.weight.sum())

Weight sum should be 260M or similar
260086951.46169


### Share of consumers living paycheck to paycheck

In [7]:
# Analysis needs financial lifestyle column as an input
# D15 is generally the code, but in case it changes here's a search function:

word_to_find = 'financial lifestyle'

for key, value in codebook.items():
    if word_to_find in value.lower():
        print(f"Word '{word_to_find}' found in key: {key}")

Word 'financial lifestyle' found in key: D15


In [8]:
#double check its the needed code:
df.D15.unique()

array(['I live paycheck to paycheck and have issues paying my bills each month.',
       'I live paycheck to paycheck but am comfortably paying my bills each month.',
       'I do not live paycheck to paycheck and have more than enough earnings to cover my bills each month.'],
      dtype=object)

In [9]:
#Two mappings: one for charts (eventually) and one for shorthand acess

p2p_mapping = {'I live paycheck to paycheck and have issues paying my bills each month.': 'Paycheck to paycheck and struggling',
           'I live paycheck to paycheck but am comfortably paying my bills each month.': 'Paycheck to paycheck but not struggling',
           'I do not live paycheck to paycheck and have more than enough earnings to cover my bills each month.': 'Not paycheck to paycheck'}

df.D15=df.D15.map(p2p_mapping)

p2p_short = {'Paycheck to paycheck and struggling':'Struggle',
           'Paycheck to paycheck but not struggling': 'P2P',
           'Not paycheck to paycheck':'NotP2P'}

df['D15_short']=df.D15.map(p2p_short)

# I could also map this into living paycheck to paycheck and not, but i already have this variable: P2P

In [10]:
print(df.D15_short.unique())
print("*"*10)
print(df.D15.unique()) # it's central to the analysis that there are no values outside the three expected values
print("*"*10)
print(f'{df.D15.isnull().sum()} null values in target column')

['Struggle' 'P2P' 'NotP2P']
**********
['Paycheck to paycheck and struggling'
 'Paycheck to paycheck but not struggling' 'Not paycheck to paycheck']
**********
0 null values


#### Share of consumers living paycheck to paycheck: Demo comparisons

In [11]:
# setting up numerator and denominator. first denominators, which are are number of people in every (sub)group
# numerator is share of people living p2p in each group
# storing results in dataframe, eventually i will be able to loop and get all of them

# Splitting sample by p2p status + quantifying the pop of each subset, gross and as a share of total
p2p_groups=df.groupby('D15') #hard coded
p2p_weight_subtotals = p2p_groups['weight'].sum()
total_weight = p2p_weight_subtotals.sum()
weight_shares = p2p_weight_subtotals / total_weight

p2p_shares = pd.DataFrame({'April 2023': round(weight_shares*100,2)}) #Date is hard coded

In [12]:
p2p_shares

,April 2023
D15,
Not paycheck to paycheck,38.90
Paycheck to paycheck and struggling,19.23
Paycheck to paycheck but not struggling,41.88


In [13]:
p2p_shares['April 2023'].sum()

100.00999999999999

In [14]:
# defining my target demographic subsets and the respective populations by split
#Age, income (3 and 5 splits), and gender. P2P not included because its the baseline split
wGens = df.groupby('Generation')['weight'].sum()
wIncome5 = df.groupby('Q_income_RB')['weight'].sum()
wIncome3 = df.groupby('income_3G')['weight'].sum()
wGender = df.groupby('L_gender')['weight'].sum()

In [15]:
# Two silly checks
wGens/1000000

Generation
Baby boomers and seniors    90.439987
Generation X                63.392723
Generation Z                31.732627
Millennials                 74.521614
Name: weight, dtype: float64

In [16]:
wIncome5/260000000 #go back and ensure these shares match the current lending analysis

Q_income_RB
$100K-$150K    0.179064
$150K-$200K    0.088617
$200K-$250K    0.040706
$250K+         0.062430
$50K-$100K     0.301647
Under $50K     0.327870
Name: weight, dtype: float64

In [17]:
# paycheck to paycheck populations withing each subgroup of each split

ppwGens=df[df['P2P']=='Paycheck to paychek'].groupby('Generation')['weight'].sum()
ppwInc5=df[df['P2P']=='Paycheck to paychek'].groupby('Q_income_RB')['weight'].sum()
ppwInc3=df[df['P2P']=='Paycheck to paychek'].groupby('income_3G')['weight'].sum()
ppwGender=df[df['P2P']=='Paycheck to paychek'].groupby('L_gender')['weight'].sum()

In [18]:
# eyeball shares of p2p by generation, make sense?
ppwGens/wGens

Generation
Baby boomers and seniors    0.502782
Generation X                0.631301
Generation Z                0.579320
Millennials                 0.738606
Name: weight, dtype: float64

In [ ]:
# Now i'm setting up a dataframe to consolidate this information
# I use crosstab

In [25]:
gen_shares=ppwGens/wGens
gen_shares=pd.DataFrame(index=gen_shares.index, data=gen_shares.values, columns=['April 2023'])

In [26]:
gen_shares

,April 2023
Generation,
Baby boomers and seniors,0.502782
Generation X,0.631301
Generation Z,0.579320
Millennials,0.738606


In [ ]:
#A) Generations
## Perform cross-tabulation with normalization
gen_cross_tab = pd.crosstab(df['Generation'], df['P2P'], normalize='index')

## Select the share of 'Paycheck to paychek' in the P2P column
gen_shares = gen_cross_tab['Paycheck to paychek']

## Turn into a dataframe (will mage with other demos/periods in the future)
gen_shares=pd.DataFrame(index=gen_shares.index, data=gen_shares.values, columns=['April 2023'])
gen_order = ['Generation Z', 'Millennials', 'Generation X','Baby boomers and seniors']
gen_shares=gen_shares.reindex(gen_order)
gen_shares=gen_shares.rename_axis('Group')

gen_shares

In [ ]:
#B) Income: 5 groups


# Perform cross-tabulation with normalization
income5_cross_tab = pd.crosstab(df['Q_income_RB'], df['P2P'], normalize='index')

# Select the share of 'Paycheck to paychek' in the P2P column
income5_shares = income5_cross_tab['Paycheck to paychek']
income5_shares=pd.DataFrame(index=income5_shares.index, data=income5_shares.values, columns=['April 2023'])

inc5_order = ['Under $50K', '$50K-$100K', '$100K-$150K','$150K-$200K','$200K-$250K','$250K+']

income5_shares=income5_shares.reindex(inc5_order)
income5_shares=income5_shares.rename_axis('Group')
income5_shares

In [ ]:
#C) Income: 3 groups

# Perform cross-tabulation with normalization
income3_cross_tab = pd.crosstab(df['income_3G'], df['P2P'], normalize='index')

# Select the share of 'Paycheck to paychek' in the P2P column
income3_shares = income3_cross_tab['Paycheck to paychek']
income3_shares=pd.DataFrame(index=income3_shares.index, data=income3_shares.values, columns=['April 2023'])

inc3_order = ['Under $50K', '$50K-$100K', '$100K+']

income3_shares=income3_shares.reindex(inc3_order)
income3_shares=income3_shares.rename_axis('Group')
income3_shares

In [ ]:
#D) Gender

# Perform cross-tabulation with normalization
gender_cross_tab = pd.crosstab(df['L_gender'], df['P2P'], normalize='index')

# Select the share of 'Paycheck to paychek' in the P2P column
gender_shares = gender_cross_tab['Paycheck to paychek']
gender_shares=pd.DataFrame(index=gender_shares.index, data=gender_shares.values, columns=['April 2023'])

gender_order = ['Female', 'Male']

gender_shares=gender_shares.reindex(gender_order)
gender_shares=gender_shares.rename_axis('Group')
gender_shares

In [ ]:
## Putting it all toghether
p2p_all_demos=pd.concat([gen_shares,income3_shares,income5_shares,gender_shares],axis=0)
p2p_all_demos